In [1]:
pip install -qq google-play-scraper

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -qq -U watermark

Note: you may need to restart the kernel to use updated packages.


In [3]:
%reload_ext watermark
%watermark -v -p pandas,matplotlib,seaborn,google_play_scraper

CPython 3.7.6
IPython 7.12.0

pandas 1.0.1
matplotlib 3.1.3
seaborn 0.10.0
google_play_scraper 0.1.1


In [4]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [5]:
app_packages = [
    'com.bigbasket.mobileapp'
]

In [6]:
app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_infos.append(info)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.19s/it]


In [7]:
def print_json(json_object):
  json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [8]:
print_json(app_infos[0])

{
  "adSupported": null,
  "androidVersion": "4.1",
  "androidVersionText": "4.1 and up",
  "appId": "com.bigbasket.mobileapp",
  "containsAds": false,
  "contentRating": "Everyone",
  "contentRatingDescription": null,
  "currency": "USD",
  "description": "bigbasket is India\u2019s leading online supermarket shopping app, trusted by 6 million+ happy customers and counting!\r\n\r\nInstall the bigbasket app now and get your groceries delivered in 25+ cities across India.\r\n\r\nShop from a vast range of 20,000+ products including farm-fresh fruits and vegetables, groceries, home & household essentials, pet care needs, organic products, beauty and hygiene, imported and gourmet & more at the best prices. Enjoy hassle-free online grocery shopping and contactless home delivery at just a click of a button.\r\n\r\nFEATURES AND SERVICES\r\n\r\nWhat to expect from us?\r\n\r\n* Wide range of products, including fresh fruits and vegetables: Choose from a vast collection of 20,000+ products across

In [11]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv('apps.csv', index=None, header=True)

In [19]:
app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 1000 if score == 3 else 800,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:18<00:00, 78.29s/it]


In [20]:
print_json(app_reviews[0])

{
  "appId": "com.bigbasket.mobileapp",
  "at": "2020-09-11 09:16:28",
  "content": "Big number game,not at all reasonable prices and the worst part is all the discounts they offer through credit card is just fake,before you pay it shows cant proceds right now.But without discount it really works good. Please dont fall into the trap,I suggest u better order it from your nearby store. On veggies and hand packed items they tag double price and then offer discount. No transparency at all.. worst experience",
  "repliedAt": "2020-09-11 09:38:55",
  "replyContent": "We are disappointed to know that your experience with us was not satisfactory. Could you please share your contact details with us on appsupport@bigbasket.com so that we can get in touch with you and resolve the issue.",
  "reviewCreatedVersion": "5.2.5",
  "reviewId": "gp:AOqpTOEq64KnW7wV7BGf2KxRP23p54_EyHv2JzwUQPyVifmsYd6E6uNT2fCB-rYJiChmB_VIT02U7RIf_GFNAQ",
  "score": 1,
  "sortOrder": "most_relevant",
  "thumbsUpCount": 80,


In [21]:
len(app_reviews)

8400

In [23]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('reviewsff.csv', index=None, header=True)